In [124]:
### Finds pure Nash equilibrium and Pareto optima

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define player class
class Player:
    def __init__(self, playerNum, payoffMatrix, playerPosition):
        self.playerNum      = playerNum
        self.payoffMatrix   = payoffMatrix
        self.playerPosition = playerPosition

In [116]:
Player1 = Player('Player1', [[3,0],[5,1]], 'row')
Player2 = Player('Player2', [[3,5],[0,1]], 'column')
#Player1 = Player('Player1', [[10,0],[12,3]], 'row')
#Player2 = Player('Player2', [[10,15],[8,3]], 'column')

In [4]:
### Find pure strategy Nash equilibrium by iteratively searching whether there's improvement in each quadrant

In [5]:
# Calculate coordinates of each quadrant. These coordinates will be used to calculate distance between cells
# Size is rows/columns in the payoff matrix.

size = 2
x = np.arange(0, size, 1)
y = np.arange(0, size, 1)
coordArr = [[(0,0) for j in y] for i in x]
for rowInd, row in enumerate((coordArr)):
    for colInd in range(len(row)):
        coordArr[rowInd][colInd] = ( rowInd, colInd )

In [99]:
# Takes coordinates of a location in the payoff matrix, the payoff matrix, and coordinates of full payoff matrix,
# and determines whether improvement is possible. If improvement is not possible in one direction, return 1. If 
# improvement is not possible in two different directions, then this is a possible pure Nash play.

def CheckNashImprovement(curPos, payoffMatrix, coordArr, playerPosition):
    # Current payoff
    curPayoff = payoffMatrix[curPos[0]][curPos[1]]
    
    orthogonal = list()
    for rowInd, row in enumerate((coordArr)):
        for colInd in range(len(row)):
            arrDiff = abs( np.subtract( coordArr[rowInd][colInd], curPos ) )
            rowDiff, colDiff = arrDiff[0], arrDiff[1]
            # The row player can only move between rows, and column player can only move between columns.
            if (playerPosition == 'row') & (rowDiff == 1) & (colDiff == 0):
                orthogonal.append(coordArr[rowInd][colInd])
            if (playerPosition == 'column') & (rowDiff == 0) & (colDiff == 1):
                orthogonal.append(coordArr[rowInd][colInd])

    # If improvement is not possible by moving to the orthogonal cell, add 1 to list. If all values are 1, then possible play
    # for one player that could constitute a Nash equilibrium
    improvement = list()
    for coord in orthogonal:
        adjPayoff = payoffMatrix[coord[0]][coord[1]]
        if adjPayoff <= curPayoff:
            improvement.append(1)
        else:
            improvement.append(0)
    
    return(improvement)

In [97]:
# Looks through lists that indicate whether improvement is possible at each position, appending all positions that are
# candidate Nash equilibria.

def FindPossibleNash( coordArr, payoffs, playerPosition ):
    
    playerPossibleNash = list()
    for rowInd, row in enumerate((payoffs)):
        for colInd in range(len(row)):
            curPos = (rowInd,colInd)
            improve = CheckNashImprovement(curPos, payoffs, coordArr, playerPosition)
            if all(x == 1 for x in improve):
                # Append the coordinates of a possible Nash equilibrium for one player.
                playerPossibleNash.append(curPos)
    
    return( playerPossibleNash )

In [123]:
# For each position, function finds all orthogonal positions, and loops through them to see whether strict improvement
# is possible. If no strict improvement is possible, the position is Pareto optimal.

def CheckParetoImprovement(curPos, coordArr, Player1, Player2):

    # Payoffs at current position
    curPayoffs = [ Player1.payoffMatrix[curPos[0]][curPos[1]],
                   Player2.payoffMatrix[curPos[0]][curPos[1]] ]

    orthogonal = list()
    for rowInd, row in enumerate((coordArr)):
        for colInd in range(len(row)):
            # Orthogonal cells are either 1 row or one column away. So orthogonal where dist == 1.
            dist = sum( abs( np.subtract( coordArr[rowInd][colInd], curPos ) ) )
            # The row player can only move between rows, and column player can only move between columns.
            if dist == 1:
                orthogonal.append( coordArr[rowInd][colInd] )

    improvement = list()
    for coord in orthogonal:  
        orthogPayoff = ( Player1.payoffMatrix[coord[0]][coord[1]],
                         Player2.payoffMatrix[coord[0]][coord[1]] )

    # If the orthogonal cell is not strictly greater than the current cell, then it is a candidate for Pareto optimality
        if ( not( \
        all( [ curPayoffs < orthogPayoff for curPayoffs, orthogPayoff \
              in zip(curPayoffs, orthogPayoff) ] ) ) ):
            improvement.append(1)
        else:
            improvement.append(0)
            
    return(improvement)

In [113]:
# Loops through each position and checks whether or not Pareto improvement is possible at each position.

def FindPareto( coordArr, Player1, Player2 ):
    paretoEquil = list()
    for rowInd, row in enumerate((coordArr)):
        for colInd in range(len(row)):
            curPos = (rowInd,colInd)
            improve = CheckParetoImprovement(curPos, coordArr, Player1, Player2)
            if all(x == 1 for x in improve):
                # Append the coordinates of a possible Nash equilibrium for one player.
                paretoEquil.append(curPos)
    return(paretoEquil)

In [122]:
# Returns two-player Nash
list( set(FindPossibleNash( coordArr, Player1.payoffMatrix, Player1.playerPosition )) & 
      set(FindPossibleNash( coordArr, Player2.payoffMatrix, Player2.playerPosition )) )

[(1, 1)]

In [120]:
FindPareto( coordArr, Player1, Player2 )

[(0, 0), (0, 1), (1, 0), (1, 1)]